In [28]:
import optuna
# optuna.delete_study(study_name='ZINC-C2C',
#     storage='sqlite:///runs/optuna.db')
study = optuna.create_study(
    sampler=optuna.samplers.TPESampler(n_ei_candidates=4),
    study_name='ZINC-C2C',
    storage='sqlite:///runs/optuna.db',
    load_if_exists=True,
    direction='minimize')
import math
import pandas as pd
distributions = {
    'transfer version' : optuna.distributions.CategoricalDistribution(['standard','simple']),
    'bn_momentum' : optuna.distributions.IntDistribution(0,3),
    'intersect_reduce (Affine Transfer)' : optuna.distributions.CategoricalDistribution(['sum','mean']),
    'hidden_channels' : optuna.distributions.IntDistribution(32,128,step=32),
    '_inner_mlp_mult' : optuna.distributions.IntDistribution(1,2),
    'num_layers' : optuna.distributions.IntDistribution(3,6),
    'patience' : optuna.distributions.IntDistribution(20,40,False,10),
}
# records = pd.read_csv('./zinc_c2c.csv')
# finished_ids = []
# for idx, run_id in enumerate(records['run #']):
#     if not math.isnan(float(records['score'][idx])):
#         params = {
#             **{key:records[key][idx] for key in ['transfer version','intersect_reduce (Affine Transfer)']},
#             **{key:-int(math.log2(10*float(records[key][idx]))) for key in ['bn_momentum']},
#             **{key:int(records[key][idx]) for key in ['hidden_channels','_inner_mlp_mult','num_layers','patience']},
#         }
#         trial = optuna.create_trial(
#             value = float(records['score'][idx]),
#             params = params,
#             user_attrs={'run_id' : run_id},
#             distributions=distributions,
#         )
#         study.add_trial(trial)
#         finished_ids.append(str(run_id))
fixed_distributions = {
    # 'transfer version' : optuna.distributions.CategoricalDistribution(['simple']),
    'bn_momentum' : optuna.distributions.IntDistribution(1,2),
    # 'intersect_reduce (Affine Transfer)' : optuna.distributions.CategoricalDistribution(['mean']),
    'hidden_channels' : optuna.distributions.IntDistribution(64,96,step=32),
    '_inner_mlp_mult' : optuna.distributions.IntDistribution(2,2),
    'num_layers' : optuna.distributions.IntDistribution(4,6),
    'patience' : optuna.distributions.IntDistribution(30,30),
}
new_trial = study.ask(fixed_distributions=fixed_distributions)
for key in distributions:
    if key == 'bn_momentum':
        print(key,':',0.1/2**new_trial._suggest(key,distributions[key]))
    else:
        print(key,':',new_trial._suggest(key,distributions[key]))
print('finished run ids:',', '.join(finished_ids))

[I 2023-10-09 12:41:00,067] A new study created in RDB with name: ZINC-C2C


transfer version : simple
bn_momentum : 0.025
intersect_reduce (Affine Transfer) : mean
hidden_channels : 64
_inner_mlp_mult : 2
num_layers : 5
patience : 30
finished run ids: 3, 4, 6, 7, 8, 9, 10, 11, 12, 14, 16, 17, 19, 20, 21, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 59


In [17]:
from optuna.visualization import plot_parallel_coordinate, plot_contour, plot_param_importances
plot_parallel_coordinate(study)
# plot_contour(study,['num_layers','intersect_reduce (Affine Transfer)'])
# plot_contour(study,['num_layers','intersect_reduce (Affine Transfer)','hidden_channels','bn_momentum'])
# plot_contour(study,['hidden_channels','bn_momentum'])
# plot_param_importances(study)